In [ ]:
import serial
import re
import mysql.connector as mysql
import time
import paho.mqtt.client as mqtt
import json
import Adafruit_DHT as dht
serial_sensor = serial.Serial("/dev/ttyACM0", 115200)
dht_sensor = dht.DHT22
db = mysql.connect(host="98.81.220.163", user="usuario", passwd="contraseña", database="BDDWINE")
cursor = db.cursor()
query_ph = 'INSERT INTO ReadPH(PH) VALUES (%s)'
query_dht = 'INSERT INTO ReadDHT22(Atemperatura, humedad) VALUES (%s, %s)'
cliente_mqtt = mqtt.Client()
cliente_mqtt.username_pw_set(username="usuario", password=" contraseña ")
cliente_mqtt.connect("98.81.220.163", 1883)
try:
    while True:
        dato = serial_sensor.readline().decode().strip()
        match = re.search(r"pH:\s*([0-9]*\.?[0-9]+)", dato, re.IGNORECASE)      
        if match:
            PH = round(float(match.group(1)), 2)
            print(f"pH = {PH}")
            cursor.execute(query_ph, (PH,))
            db.commit()
            print("Lectura de pH insertada en la base de datos")
            datos_dict = {'PH': PH}
            datos_json = json.dumps(datos_dict)
            cliente_mqtt.publish("readPH", datos_json)
            print("Datos de pH publicados en MQTT")
        else:
            print("Error al leer el sensor pH")
        humidity, temperature = dht.read_retry(dht_sensor, 4)
        if humidity is not None and temperature is not None:
            temperature = round(temperature, 2)
            humidity = round(humidity, 2)
            print(f"Temperatura = {temperature} oC, Humedad = {humidity} %")
            cursor.execute(query_dht, (temperature, humidity))
            db.commit()
            print("Lectura de DHT22 insertada en la base de datos")
            datos_dict = {
                'temperature': temperature,
                'humidity': humidity
            }
            datos_json = json.dumps(datos_dict)
            cliente_mqtt.publish("readDHT22", datos_json)
            print("Datos de DHT22 publicados en MQTT")
        else:
            print("Error al leer el sensor DHT22")
        time.sleep(14400)
except KeyboardInterrupt:
    print("Fin de programa")
